In [1]:
from pyspark import SparkContext #Library
sc = SparkContext("local[*]", "temp") #Start a spart context


In [2]:
posts_lines = sc.textFile("file:///home/vagrant/miniprojects/spark/part/allPosts/*.gz") #create an RDD from file
print posts_lines
posts_lines.count()
#posts_lines.take(8)

MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2


212990

In [3]:
users_lines = sc.textFile("file:///home/vagrant/miniprojects/spark/part/allUsers/*.gz")
print users_lines
users_lines.count()

MapPartitionsRDD[4] at textFile at NativeMethodAccessorImpl.java:-2


100425

In [4]:
#Import parsing library.
import xml.etree.ElementTree as ET
import numpy as np


In [5]:
def parseUser(line):
    if '  <row' in line:
        try:
            root = ET.fromstring(line.encode('utf8'))
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "Id" and "Reputation" in root.attrib:
                return(root.attrib["Id"], int(root.attrib["Reputation"]))
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")
            

In [6]:
users = users_lines.map(parseUser).filter(lambda x: x!= 'Empty')
print users.count()
print users.take(99)

50320
[('70185', 1), ('70186', 6), ('70187', 1), ('70188', 1), ('70189', 101), ('70190', 1), ('70191', 1), ('70192', 1), ('70193', 1), ('70194', 11), ('70195', 1), ('70196', 6), ('70197', 18), ('70198', 1), ('70199', 51), ('70200', 11), ('70201', 1), ('70202', 101), ('70203', 116), ('70204', 103), ('70205', 6), ('70206', 101), ('70207', 1), ('70208', 101), ('70211', 1), ('70212', 1), ('70213', 1), ('70214', 1), ('70215', 11), ('70216', 1), ('70217', 1), ('70218', 1), ('70219', 101), ('70220', 1), ('70221', 1), ('70222', 1), ('70223', 1), ('70224', 6), ('70225', 1), ('70226', 11), ('70227', 101), ('70228', 1), ('70229', 1), ('70230', 1), ('70231', 1), ('70232', 1), ('70233', 1), ('70234', 1), ('70235', 1), ('70236', 1), ('70237', 3), ('70238', 1), ('70239', 1), ('70240', 1), ('70241', 1), ('70242', 1), ('70243', 6), ('70244', 101), ('70245', 1), ('70246', 1), ('70247', 1), ('70248', 1), ('70249', 1), ('70250', 4), ('70251', 1), ('70252', 1), ('70253', 101), ('70254', 1), ('70255', 1), (

In [7]:
def parsePost(line):
    if '  <row' in line:
        try:
            root = ET.fromstring(line.encode('utf8'))
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "OwnerUserId" in root.attrib:
                try:
                    owner = root.attrib["OwnerUserId"]
                except:
                    pass
                    return ("Empty")
                return(owner, 1)
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")
            
        

In [8]:
posts = posts_lines.map(parsePost).filter(lambda x: x!= 'Empty')\
        .reduceByKey(lambda x, y: x+y)
print posts
print posts.take(40)
posts.count()


PythonRDD[12] at RDD at PythonRDD.scala:43
[('23994', 1), ('61518', 2), ('35549', 1), ('69951', 1), ('29446', 1), ('11549', 3), ('21631', 1), ('60913', 1), ('17297', 1), ('11384', 8), ('55294', 1), ('43270', 1), ('10897', 1), ('5987', 1), ('22267', 2), ('48893', 2), ('3993', 1), ('61176', 1), ('56583', 2), ('53900', 1), ('25065', 1), ('56236', 2), ('12703', 1), ('61195', 1), ('43185', 2), ('35430', 1), ('42457', 3), ('31915', 1), ('37734', 2), ('5857', 1), ('55587', 1), ('69554', 1), ('291', 4), ('3773', 3), ('60136', 1), ('46484', 1), ('69370', 1), ('21108', 1), ('56053', 1), ('40605', 2)]


26890

In [9]:
joint = posts.join(users).map(lambda x: (x[1][0], (x[1][1], 1.)))\
        .reduceByKey(lambda x,y :np.add(x,y)).map(lambda x: (x[0], x[1][0]/x[1][1]))\
        .sortByKey(False).collect()
print joint[:100]

[(2325, 92624.0), (1663, 47334.0), (1287, 100976.0), (1018, 46907.0), (965, 23102.0), (695, 27599.0), (570, 22706.0), (558, 25406.0), (495, 9294.0), (494, 23610.0), (469, 10728.0), (452, 32283.0), (424, 16854.0), (419, 17719.0), (395, 14100.0), (390, 20315.0), (369, 19312.0), (363, 6149.0), (350, 9047.0), (345, 14768.0), (343, 13557.0), (339, 11795.0), (338, 10045.0), (304, 16131.0), (301, 6352.0), (297, 20133.0), (292, 10552.0), (290, 8285.5), (287, 11083.0), (282, 10383.0), (277, 11830.0), (269, 7729.0), (268, 11989.0), (267, 7971.0), (265, 7765.0), (257, 13078.0), (248, 7608.0), (247, 12496.5), (239, 1.0), (234, 11307.5), (228, 11662.0), (226, 5775.0), (218, 5849.0), (211, 7552.0), (208, 6208.0), (202, 9530.0), (195, 9619.0), (193, 6682.0), (188, 12098.0), (187, 8013.0), (185, 4149.0), (184, 5762.0), (177, 5042.0), (173, 10394.0), (168, 7725.0), (167, 3957.0), (165, 6694.0), (164, 1544.0), (163, 6888.0), (161, 6367.0), (159, 7116.0), (157, 6040.0), (156, 4086.6666666666665), (155, 4

In [ ]:
[(118, 3736.5)] * 100